In [21]:
import numpy as np
import pandas as pd

### Aufgabe 1
Format für products: Name, Preis, Gewicht

In [22]:
products = np.array(["Edelstein","Epoxidharz","Kupfer"])
profits = np.array([30, 10, 100])
weights = np.array([2, 1, 6])



In [23]:
# Jedes Szenario wird durchlaufen (BruteForce)
def check_constraints(ed, ku, w_total, u_total, w_max, u_max):
    return ku*2 <= ed and w_total <= w_max and u_total <= u_max

def optimize(u_max, w_max):
    results = []
    u_max = u_max
    for ed in range(u_max):
        for ep in range(u_max):
            for ku in range(u_max):
                u_total = ed + ep + ku
                w_total = weights[0] * ed + weights[1] * ep + weights[2] * ku
                if check_constraints(ed, ku, w_total, u_total, w_max, u_max):
                    profit = profits[0] * ed + profits[1] * ep + profits[2] * ku
                    results.append({
                        "Gewinn": profit,
                        "Gewicht": w_total,
                        "Einheiten": u_total,
                        "Edelstein": ed,
                        "Epoxidharz": ep,
                        "Kupfer": ku
                    })

    return results


In [24]:
max_units = 15
max_weight = 56
optimized_products = optimize(max_units, max_weight)
df = pd.DataFrame(optimized_products)
# Optimized_Products wird nach Gewinn sortiert und es werden nur die "Top 10" angezeigt
sorted_df = df.sort_values(by="Gewinn", ascending=False).reset_index(drop=True).loc[:10]

In [25]:
print(sorted_df)

    Gewinn  Gewicht  Einheiten  Edelstein  Epoxidharz  Kupfer
0      800       50         15         10           0       5
1      730       46         15         11           0       4
2      710       45         15         10           1       4
3      700       44         14         10           0       4
4      690       44         15          9           2       4
5      680       43         14          9           1       4
6      670       42         13          9           0       4
7      670       43         15          8           3       4
8      660       42         14          8           2       4
9      660       42         15         12           0       3
10     650       41         13          8           1       4
